# Clase Práctica #16 (Compilación)

En esta clase estaremos implementado un mecanismo para transformar _ASTs_ de un lenguaje origen hacia un lenguaje intermedio. Trabajaremos con el lenguaje estudiado en las clases anteriores y lo transformaremos al lenguaje _CIL_. Recordemos que _CIL_ es un lenguaje intermedio cuyo único objetivo es proveer una representación cómoda para transitar entre el lenguaje origen y el destino.

Pasemos a importar el trabajo de las clases anteriores.

### Análisis Lexicográfico y Sintáctico

In [1]:
import cmp.nbpackage
import cmp.visitor as visitor

from cp13 import G, text
from cp13 import Node, ProgramNode, DeclarationNode, ExpressionNode
from cp13 import ClassDeclarationNode, FuncDeclarationNode, AttrDeclarationNode
from cp13 import VarDeclarationNode, AssignNode, CallNode
from cp13 import AtomicNode, BinaryNode
from cp13 import ConstantNumNode, VariableNode, InstantiateNode, PlusNode, MinusNode, StarNode, DivNode
from cp13 import FormatVisitor, tokenize_text, pprint_tokens

from cmp.tools.parsing import LR1Parser
from cmp.evaluation import evaluate_reverse_parse

importing Jupyter notebook from cp13.ipynb
Non-Terminals:
	<program>, <class-list>, <def-class>, <feature-list>, <def-attr>, <def-func>, <param-list>, <param>, <expr-list>, <expr>, <arith>, <term>, <factor>, <atom>, <func-call>, <arg-list>
Terminals:
	class, let, def, print, ;, :, ,, ., (, ), {, }, =, +, -, *, /, id, int, new
Productions:
	[<program> -> <class-list>, <class-list> -> <def-class> <class-list>, <class-list> -> <def-class>, <def-class> -> class id { <feature-list> }, <def-class> -> class id : id { <feature-list> }, <feature-list> -> <def-attr> <feature-list>, <feature-list> -> <def-func> <feature-list>, <feature-list> -> e, <def-attr> -> id : id ;, <def-func> -> def id ( <param-list> ) : id { <expr-list> }, <param-list> -> <param>, <param-list> -> <param> , <param-list>, <param> -> id : id, <expr-list> -> <expr> ; <expr-list>, <expr-list> -> <expr> ;, <expr> -> let id : id = <expr>, <expr> -> let id = <expr>, <expr> -> <arith>, <arith> -> <arith> + <term>, <arith> -> <arit

### Análisis Semántico (Recolección y Construcción de Tipos)

In [2]:
from cmp.semantic import SemanticError
from cmp.semantic import Attribute, Method, Type
from cmp.semantic import VoidType, ErrorType, IntType
from cmp.semantic import Context

from cp14 import TypeCollector, TypeBuilder

importing Jupyter notebook from cp14.ipynb
=================== TEXT ======================

class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}

================== TOKENS =====================
class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id ;
    }
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
}
$
=================== PARSE =====================
<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <arith> + <term>
<e

<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <arith> + <term>
<expr> -> <arith>
<expr-list> -> <expr> ;
<def-func> -> def id ( <param-list> ) : id { <expr-list> }
<def-attr> -> id : id ;
<def-attr> -> id : id ;
<feature-list> -> e
<feature-list> -> <def-attr> <feature-list>
<feature-list> -> <def-attr> <feature-list>
<feature-list> -> <def-func> <feature-list>
<feature-list> -> <def-attr> <feature-list>
<def-class> -> class id { <feature-list> }
<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> int
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<expr> -> <arith>
<expr> -> let id : id = <expr>
<atom> -> new id ( )
<factor> -> <atom>
<atom> -> int
<factor> -> <atom>
<term> -> <factor>
<ar

AssertionError: 

### Análisis Semántico (Chequeo de tipos)

In [ ]:
from cmp.semantic import Scope, VariableInfo
from cp15 import TypeChecker

## Transformación a CIL

Partiendo de un AST semánticamente anotado (lo hicimos en las clases anteriores) queremos obtener un AST en otro lenguaje. Como es usual, realizaremos un recorrido por el AST original usando el patrón `visitor` pero esta vez pretendemos generar como salida un nuevo AST. Esto se traduce en devolver en el llamado principal de `visit`, el nodo que representa la raíz del nuevo lenguaje.

El módulo `cmp.cil` provee una implementación de todos los nodos de CIL necesarios para transformar nuestro lenguaje. Además, se provee la implementación de `BaseCOOLToCILVisitor` que usaremos como base para realizar el recorrido. Esta clase contiene métodos auxiliares para facilitar la generación del nuevo AST.

- Las variables de instancia `dottypes`, `dotdata` y `dotcode` almacenan los nodos correspondientes a las secciones `.TYPES`, `.DATA` y `.CODE` respectivamente de un programa en CIL.
- Las variables `current_type` y `current_method` almacenan instancias de `Type` y `Method` respectivamente.
- La variable `current_function` almacena el nodo `cil.FunctionNode` que está en proceso de construcción (estos nodos pertenecen a la sección `.CODE`).
- Para definir variables locales e instrucciones dentro de `current_function` se usan las funciones auxiliares `register_local` y `register_instruction` respectivamente.
- En caso de que se necesite definir una variable no declarada por el programador (para guardar resultados intermedios), el método `define_internal_local` provee un mecanismo para hacerlo.
- Los métodos `register_function` y `register_type` almacenan instancias de `cil.FunctionNode` y `cil.TypeNode` en las variables `dotcode` y `dottypes` respectivamente.

In [ ]:
class BaseCOOLToCILVisitor:
    def __init__(self, context):
        self.dottypes = []
        self.dotdata = []
        self.dotcode = []
        self.current_type = None
        self.current_method = None
        self.current_function = None
        self.context = context
    
    @property
    def params(self):
        return self.current_function.params
    
    @property
    def localvars(self):
        return self.current_function.localvars
    
    @property
    def instructions(self):
        return self.current_function.instructions
    
    def register_local(self, vinfo):
        vinfo.name = f'local_{self.current_function.name[9:]}_{vinfo.name}_{len(self.localvars)}'
        local_node = cil.LocalNode(vinfo.name)
        self.localvars.append(local_node)
        return vinfo.name

    def define_internal_local(self):
        vinfo = VariableInfo('internal', None)
        return self.register_local(vinfo)

    def register_instruction(self, instruction):
        self.instructions.append(instruction)
        return instruction
    
    def to_function_name(self, method_name, type_name):
        return f'function_{method_name}_at_{type_name}'
    
    def register_function(self, function_name):
        function_node = cil.FunctionNode(function_name, [], [], [])
        self.dotcode.append(function_node)
        return function_node
    
    def register_type(self, name):
        type_node = cil.TypeNode(name)
        self.dottypes.append(type_node)
        return type_node

    def register_data(self, value):
        vname = f'data_{len(self.dotdata)}'
        data_node = cil.DataNode(vname, value)
        self.dotdata.append(data_node)
        return data_node

### Implementación

Pasemos a implementar un `visitor` concretamente. Para simplificar la implementación, asuma que el acceso (`VariableNode`) y asignación (`AssignNode`) de variables ocurre solo sobre variables locales. Realmente sería necesario desambiguar en cuales casos es un acceso a variable y en cuales un acceso a atributo. Esto queda propuesto como estudio individual.

In [ ]:
import cmp.cil as cil

class MiniCOOLToCILVisitor(BaseCOOLToCILVisitor):
    @visitor.on('node')
    def visit(self, node):
        pass
    
    @visitor.when(ProgramNode)
    def visit(self, node, scope):
        ######################################################
        # node.declarations -> [ ClassDeclarationNode ... ]
        ######################################################
        
        self.current_function = self.register_function('entry')
        instance = self.define_internal_local()
        result = self.define_internal_local()
        main_method_name = self.to_function_name('main', 'Main')
        self.register_instruction(cil.AllocateNode('Main', instance))
        self.register_instruction(cil.ArgNode(instance))
        self.register_instruction(cil.StaticCallNode(main_method_name, result))
        self.register_instruction(cil.ReturnNode(0))
        self.current_function = None
        
        for declaration, child_scope in zip(node.declarations, scope.children):
            self.visit(declaration, child_scope)

        return cil.ProgramNode(self.dottypes, self.dotdata, self.dotcode)
    
    @visitor.when(ClassDeclarationNode)
    def visit(self, node, scope):
        ####################################################################
        # node.id -> str
        # node.parent -> str
        # node.features -> [ FuncDeclarationNode/AttrDeclarationNode ... ]
        ####################################################################
        
        self.current_type = self.context.get_type(node.id)
        
        # Your code here!!! (Handle all the .TYPE section)
          
        func_declarations = (f for f in node.features if isinstance(f, FuncDeclarationNode))
        for feature, child_scope in zip(func_declarations, scope.children):
            self.visit(feature, child_scope)
                
        self.current_type = None
                
    @visitor.when(FuncDeclarationNode)
    def visit(self, node, scope):
        ###############################
        # node.id -> str
        # node.params -> [ (str, str) ... ]
        # node.type -> str
        # node.body -> [ ExpressionNode ... ]
        ###############################
        
        self.current_method = self.current_type.get_method(node.id)
        
        # Your code here!!! (Handle PARAMS)
        
        for instruction, child_scope in zip(node.body, scope.children):
            value = self.visit(instruction, child_scope)
        # Your code here!!! (Handle RETURN)
        
        self.current_method = None

    @visitor.when(VarDeclarationNode)
    def visit(self, node, scope):
        ###############################
        # node.id -> str
        # node.type -> str
        # node.expr -> ExpressionNode
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(AssignNode)
    def visit(self, node, scope):
        ###############################
        # node.id -> str
        # node.expr -> ExpressionNode
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(CallNode)
    def visit(self, node, scope):
        ###############################
        # node.obj -> AtomicNode
        # node.id -> str
        # node.args -> [ ExpressionNode ... ]
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(ConstantNumNode)
    def visit(self, node, scope):
        ###############################
        # node.lex -> str
        ###############################
        
        # Your code here!!! (Pretty easy ;-))
        pass

    @visitor.when(VariableNode)
    def visit(self, node, scope):
        ###############################
        # node.lex -> str
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(InstantiateNode)
    def visit(self, node, scope):
        ###############################
        # node.lex -> str
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(PlusNode)
    def visit(self, node, scope):
        ###############################
        # node.left -> ExpressionNode
        # node.right -> ExpressionNode
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(MinusNode)
    def visit(self, node, scope):
        ###############################
        # node.left -> ExpressionNode
        # node.right -> ExpressionNode
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(StarNode)
    def visit(self, node, scope):
        ###############################
        # node.left -> ExpressionNode
        # node.right -> ExpressionNode
        ###############################
        
        # Your code here!!!
        pass

    @visitor.when(DivNode)
    def visit(self, node, scope):
        ###############################
        # node.left -> ExpressionNode
        # node.right -> ExpressionNode
        ###############################
        
        # Your code here!!!
        pass
        
    # ======================================================================

### Pipeline

Actualicemos el método `run_pipeline` para incluir esta nueva fase. Con eso deberíamos completar una línea de ejecución que, partiendo desde el programa en texto plano y la gramática, produzca una representación en CIL.

In [ ]:
from cp15 import run_pipeline as deprecated_pipeline
from cmp.cil import get_formatter

formatter = get_formatter()

def run_pipeline(G, text):
    ast, errors, context, scope = deprecated_pipeline(G, text)
    print('============= TRANSFORMING TO CIL =============')
    cool_to_cil = MiniCOOLToCILVisitor(context)
    cil_ast = cool_to_cil.visit(ast, scope)
    formatter = get_formatter()
    print(formatter(cil_ast))
    return ast, errors, context, scope, cil_ast

### Programa #1

El siguiente programa no debería contener errores.

In [ ]:
text = '''
class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : int ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}
'''

text_cil_no_attr = '''.TYPES
type A {
	attribute a
	attribute b

	method suma: function_suma_at_A
}
type B {
	attribute a
	attribute b
	attribute c

	method suma: function_suma_at_A
	method f: function_f_at_B
}

.DATA


.CODE
function entry {
	

	LOCAL local__internal_0
	LOCAL local__internal_1

	local__internal_0 = ALLOCATE Main
	ARG local__internal_0
	local__internal_1 = CALL function_main_at_Main
	RETURN 0
}
function function_suma_at_A {
	PARAM self
	PARAM a
	PARAM b

	LOCAL local_suma_at_A_internal_0

	local_suma_at_A_internal_0 = a + b
	RETURN local_suma_at_A_internal_0
}
function function_f_at_B {
	PARAM self
	PARAM d
	PARAM a

	LOCAL local_f_at_B_f_0

	local_f_at_B_f_0 = 8
	RETURN 
}'''

if __name__ == '__main__':
    ast, errors, context, scope, cil_ast = run_pipeline(G, text)    
    assert formatter(cil_ast) == text_cil_no_attr

## Propuestas

- Maneje el acceso y asignación a atributos (`GETATTR` y `SETATTR`).
- Implemente un intérprete de `CIL`.